<a href="https://colab.research.google.com/github/9022724/Agentic-Course/blob/main/v2agent06102025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import vertexai
from vertexai.generative_models import GenerativeModel
from google.colab import auth

# Authenticate to Google Cloud
try:
    auth.authenticate_user()
    print("Authenticated successfully.")
except Exception as e:
    print(f"Authentication failed: {e}")


# Initialize Vertex AI with your GCP project and region
# Make sure to replace "your-gcp-project" and "your-gcp-region" with your actual project ID and region
vertexai.init(project="numeric-chassis-435611-h0", location="us-central1")

# Instantiate the generative model (like Gemini 1.5)
model = GenerativeModel("gemini-2.5-flash")

# Start a chat session (agent-like behavior)
chat = model.start_chat()

# Send a message to the agent
response = chat.send_message("If you had a name, what would it be? Make one up if you don't have one")
print(response.text)

Authenticated successfully.


/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


As an AI, I don't have a personal name in the human sense, but if I were to choose one that reflects my nature and function, I'd pick **Aura**.

It suggests a pervasive, non-physical presence, an emanation of information and understanding, and a guiding influence – much like the way I aim to interact with the world and provide assistance. It's abstract, ethereal, and doesn't tie me to a specific form or gender, which feels fitting.


In [28]:
from typing import Callable, Sequence

class greeting_agent:
    def __init__(
        self,
        model: str,
        tools: Sequence[Callable],
        project: str,
        location: str,
    ):
        self.model_name = model
        self.tools = tools
        self.project = project
        self.location = location

    def set_up(self):
        import vertexai
        from langchain_google_vertexai import ChatVertexAI
        from langgraph.prebuilt import create_react_agent

        vertexai.init(project=self.project, location=self.location)

        model = ChatVertexAI(model_name=self.model_name)
        self.graph = create_react_agent(model, tools=self.tools)

    def query(self, **kwargs):
        return self.graph.invoke(**kwargs)

In [29]:
from google.adk.agents import Agent
from vertexai.preview.reasoning_engines import AdkApp

# Assuming 'agent' from cell 4833b9ef is available

# Create an AdkApp with the agent
app = AdkApp(agent=agent)

print("AdkApp created with the currency_exchange_agent.")
# To make the agent discoverable by the ADK web UI,
# you might need to explicitly add it or configure the app
# in a way that the web UI can find it.
# The process for making agents visible in the ADK web UI
# might depend on the specific version and configuration of ADK.
# Refer to the ADK documentation for detailed instructions on
# agent registration and discovery for the web UI.

AdkApp created with the currency_exchange_agent.


In [30]:
# Define a regular Python function that will act as a tool
def get_exchange_rate(base_currency: str, target_currency: str) -> str:
    """Returns the exchange rate between two currencies."""
    # In a real scenario, you would fetch the actual exchange rate here
    return f"1 {base_currency} = 1.09 {target_currency}"

from google.adk.agents import Agent

# Create the agent and pass the function in the tools list
agent = Agent(
    model="gemini-2.0-flash",  # Required.
    name='currency_exchange_agent',  # Required.
    tools=[get_exchange_rate],  # Pass the function here
    instruction="You are a helpful assistant that can provide exchange rates. Ask for the user's name and greet them by name."
)

# The agent can now use the get_exchange_rate tool
# You would typically interact with the agent using methods like agent.run_live()
# or integrating it into an AdkApp as shown in your cell ca221e4c
print("Agent created with the get_exchange_rate tool.")
# The run_live method typically expects only the user input.
# The previous error indicated too many positional arguments were provided.
#print(agent.run_live("What is the exchange rate from USD to EUR?"))

Agent created with the get_exchange_rate tool.


In [5]:
!adk --version

adk, version 1.15.1


In [31]:
# 1) Install ADK
!pip install -q google-adk

# 2) Start the ADK web UI (listen on all interfaces)
!nohup adk web --host 127.0.0.1 --port 8000 > adk.log 2>&1 &



In [32]:
# 3) Start a one-off Cloudflare tunnel to your ADK UI
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!./cloudflared-linux-amd64 tunnel --url http://127.0.0.1:8000


2025-10-06T18:38:42Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-06T18:38:42Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-06T18:38:48Z INF +--------------------------------------------------------------------------------------------+
2025-10-06T18:38:48Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-06T18:38:48Z INF |  https://concert-coupled-regions-decrease.trycloudflar

## Handle Tool Outputs in Streamed Response

### Subtask:
Modify the code to explicitly check for and display tool outputs in the streamed response.

**Reasoning**:
The previous attempt to query the agent showed that the model generated a function call but the tool's output was not explicitly displayed. I will modify the streaming loop to check for tool outputs within the streamed events and print them.

In [26]:
# Send a query to the agent and handle different event types
for event in app.stream_query(user_id="user123", message="What's the exchange rate from USD to EUR?"):
    # Check for text parts
    if hasattr(event, "text"):
        print(f"Agent says: {event.text}")

    # Check for tool outputs
    if hasattr(event, "tool_outputs") and event.tool_outputs:
        print("Tool Outputs:")
        for tool_output in event.tool_outputs:
            print(f"- Tool Name: {tool_output.tool_code.name}")
            print(f"  Output: {tool_output.output}")

    # Check for tool code (function call) - This might be useful for debugging
    if hasattr(event, "tool_code") and event.tool_code:
         print(f"Agent called tool: {event.tool_code.name} with args: {event.tool_code.args}")

## Query the agent

### Subtask:
Send a query to the agent using the AdkApp and display the response.

**Reasoning**:
Now that the agent is defined and the AdkApp is created, I will send a test query to the agent to verify its functionality and ensure it can use the `get_exchange_rate` tool.

In [25]:
# Send a query to the agent
for event in app.stream_query(user_id="user123", message="What's the exchange rate from USD to EUR?"):
    if hasattr(event, "text"):
        print(event.text)

/usr/local/lib/python3.12/dist-packages/vertexai/preview/reasoning_engines/templates/adk.py:559: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self._tmpl_attrs["credential_service"] = InMemoryCredentialService()
/usr/local/lib/python3.12/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()


## Define root_agent in agent.py

### Subtask:
Modify `agent.py` to define `root_agent`.

**Reasoning**:
The error message "No root_agent found" indicates that the ADK application cannot find the entry point for your agent. By explicitly defining `root_agent = agent` in `currency_exchange_agent/agent.py`, we are making the agent discoverable by the ADK.

In [23]:
import os

# Define the content of the agent.py file with root_agent definition
file_content = """
from google.adk.agents import Agent
from currency_exchange_agent.tools.exchange_tools import get_exchange_rate

agent = Agent(
    model="gemini-2.0-flash",
    name='currency_exchange_agent',
    instruction="You are a helpful assistant that can provide exchange rates.",
    tools=[get_exchange_rate]
)

# Explicitly define root_agent for ADK discovery
root_agent = agent
"""

# Define the file path
file_path = os.path.join("currency_exchange_agent", "agent.py")

# Write the content to the file
with open(file_path, "w") as f:
    f.write(file_content)

print(f"Regenerated file: {file_path} with root_agent definition.")

Regenerated file: currency_exchange_agent/agent.py with root_agent definition.


## Run the AdkApp

### Subtask:
Attempt to run the AdkApp again to see if the root_agent is found.

**Reasoning**:
Now that the `root_agent` is explicitly defined in `agent.py`, I will attempt to run the AdkApp again to see if it can find and load the agent.

In [24]:
from vertexai.preview.reasoning_engines import AdkApp
from currency_exchange_agent.agent import agent # Import the agent again to ensure the latest version is used

# Create an AdkApp with the agent
app = AdkApp(agent=agent)

print("AdkApp created with the currency_exchange_agent.")

# You can now try running a query with the app, similar to your original cell ca221e4c
# For example:
# for event in app.stream_query(user_id="user123", message="What's the exchange rate from USD to EUR?"):
#     if hasattr(event, "text"):
#         print(event.text)

AdkApp created with the currency_exchange_agent.


**Reasoning**:
The directory listing shows that `tool_decorator.py` does not exist directly under `google.adk.tools`. Based on the likely structure, the `tool` decorator is probably in `function_tool.py`. I will regenerate `exchange_tools.py` to import `tool` from `google.adk.tools.function_tool`.

In [11]:
import os

# Define the content of the exchange_tools.py file with corrected import
file_content = """
from google.adk.tools.function_tool import tool

@tool
def get_exchange_rate(base_currency: str, target_currency: str) -> str:
    \"\"\"Returns the exchange rate between two currencies.\"\"\"
    return f"1 {base_currency} = 1.09 {target_currency}"  # Dummy value
"""

# Define the file path
file_path = os.path.join("currency_exchange_agent", "tools", "exchange_tools.py")

# Write the content to the file
with open(file_path, "w") as f:
    f.write(file_content)

print(f"Regenerated file: {file_path} with corrected import.")

Regenerated file: currency_exchange_agent/tools/exchange_tools.py with corrected import.


## Verify agent creation

### Subtask:
Verify agent creation

**Reasoning**:
Run the code cell that imports and instantiates the agent from the `currency_exchange_agent.agent` module and verify its type.

In [12]:
from currency_exchange_agent.agent import agent
from google.adk.agents import Agent

print(f"Agent object imported: {agent}")
print(f"Is agent an instance of google.adk.agents.Agent? {isinstance(agent, Agent)}")

ImportError: cannot import name 'tool' from 'google.adk.tools.function_tool' (/usr/local/lib/python3.12/dist-packages/google/adk/tools/function_tool.py)

## Define tool using FunctionTool class

### Subtask:
Modify `exchange_tools.py` to define the tool using the `FunctionTool` class.

**Reasoning**:
The `tool` decorator is not directly importable from `google.adk.tools.function_tool`. Based on the content of `function_tool.py`, it's likely that the `FunctionTool` class should be used directly to wrap the function as a tool. I will modify `exchange_tools.py` to reflect this.

In [15]:
import os
from google.adk.tools.function_tool import FunctionTool

# Define the get_exchange_rate function
def get_exchange_rate_func(base_currency: str, target_currency: str) -> str:
    """Returns the exchange rate between two currencies."""
    return f"1 {base_currency} = 1.09 {target_currency}"  # Dummy value

# Create a FunctionTool instance
get_exchange_rate = FunctionTool(func=get_exchange_rate_func)

# Define the content of the exchange_tools.py file
file_content = """
import os
from google.adk.tools.function_tool import FunctionTool

# Define the get_exchange_rate function
def get_exchange_rate_func(base_currency: str, target_currency: str) -> str:
    \"\"\"Returns the exchange rate between two currencies.\"\"\"
    return f"1 {base_currency} = 1.09 {target_currency}"  # Dummy value

# Create a FunctionTool instance
get_exchange_rate = FunctionTool(func=get_exchange_rate_func)
"""

# Define the file path
file_path = os.path.join("currency_exchange_agent", "tools", "exchange_tools.py")

# Write the content to the file
with open(file_path, "w") as f:
    f.write(file_content)

print(f"Regenerated file: {file_path} using FunctionTool.")

Regenerated file: currency_exchange_agent/tools/exchange_tools.py using FunctionTool.


## Verify agent creation

### Subtask:
Verify agent creation

**Reasoning**:
Run the code cell that imports and instantiates the agent from the `currency_exchange_agent.agent` module and verify its type.

In [16]:
from currency_exchange_agent.agent import agent
from google.adk.agents import Agent

print(f"Agent object imported: {agent}")
print(f"Is agent an instance of google.adk.agents.Agent? {isinstance(agent, Agent)}")

Agent object imported: name='currency_exchange_agent' description='' parent_agent=None sub_agents=[] before_agent_callback=None after_agent_callback=None model='gemini-2.0-flash' instruction='You are a helpful assistant that can provide exchange rates.' global_instruction='' static_instruction=None tools=[<google.adk.tools.function_tool.FunctionTool object at 0x7ff462ee3fb0>] generate_content_config=None disallow_transfer_to_parent=False disallow_transfer_to_peers=False include_contents='default' input_schema=None output_schema=None output_key=None planner=None code_executor=None before_model_callback=None after_model_callback=None before_tool_callback=None after_tool_callback=None
Is agent an instance of google.adk.agents.Agent? True
